# HiFi-GAN: Generative Adversarial Networks for Efficient and High Fidelity Speech Synthesis

- **HiFi-GAN**: **Mel-spectrogram**을 **Waveform**으로 변환하는 Generative Adversarial Network(**GAN**) 기반의 딥러닝 **Vocoder** 모델

# 0. Abstract

- **HiFi-GAN**은 다음과 같은 특징을 지닌 모델이다.
    1. 인간의 목소리와 유사한 수준의 MOS(Mean Opinion Score)를 보여준다.
    2. **음성 합성 속도**가 **WaveNet(MoL)**보다 **GPU**에서 최대 **1,186배 빠르고**, **CPU**에서 최대 **13.44배 빠르다.**
    3. **파라미터 수**도 최소 **0.92M**으로, **WaveNet(MoL)**의 **24.73M**에 비해 **매우 적다.**

# 1. Introduction

- **HiFi-GAN**은 **Mel-spectrogram**을 **Waveform**으로 변환하는 모델이다.
<img src="1.png" width=70% height=70%>

- **Autoregressive(AR) Model**: 이전 시간 단계의 정보로 현재 시간 단계를 예측하는 모델
    - 예) **WaveNet**은 **AR Model**로, 딥러닝 기반으로 음성 합성을 성공적으로 해낸 최초의 모델 중 하나이다.
    - **단점**: 한 샘플씩 생성하다보니 **생성 속도가 매우 느리다.**
        - 이를 극복하기 위해 **Flow-based generative Model**이 등장한다.

- **Flow-based Generative Model**: 일대일 변환 함수를 이용하여 **입력 데이터**와 **잠재 변수** 간의 **확률 분포**를 예측하는 모델
    - 예1) **Parallel WaveNet**: WaveNet과 동일한 성능을 내면서 1,000배 빠른 음성 합성 속도를 보여준다. 
    - 예2) **WaveGlow**: teacher forcing이 아니며, maximum likelihood estimation을 사용한다. WaveNet보다 좋은 품질의 음성을 합성한다.
    - **단점**: 너무 깊은 층수의 구조로 인해 **많은 파라미터 수**를 갖는다.
        - 이를 극복하기 위해 **Generative Adversarial Networks(GANs)**를 이용한 음성 합성 모델이 등장한다.

- **Generative Adversarial Networks(GANs)**: **생성자(generator)**와 **판별자(discriminator)**의 **경쟁적인 학습**을 이용하는 딥러닝 생성 모델
    - 예1) **MelGAN**: CPU에서도 음성 합성이 가능할 정도로 압축된 모델이다.
    - 예2) **GAN-TTS**: Mel-spectrogram 없이 여러 판별자를 이용해서 waveform을 생성한다. Parallel WaveNet에 비해 연산 횟수가 적다.
    - **단점**: AR Model과 Flow-based Generative Model에 비해 **음성 합성 품질이 낮다.**

- **HiFi-GAN**: **GAN** 기반 음성 합성 모델
    - **빠른 생성 속도**: **AR Model**보다 GPU에서 **1,186배 빠르다.**
    - **적은 파라미터 수**: 최소 **0.92M** 파라미터 수를 갖는다.
    - **높은 음성 합성 품질**: AR Model, Flow-based Generative Model에 비해 높은 음성 합성 품질을 보인다.

# 2. Architecture

## 2.1. Overview

- HiFi-GAN은 **1개의 Generator**와 **2개의 Discriminator**를 갖는다.
<img src="2.png" width=70% height=70%>

    - **Generator**: Mel-spectrogram을 Waveform으로 변환하는 역할을 한다.
    - **Multi-Period Discriminator(MPD)**: Speech audio는 다양한 주기의 사인파로 구성되어 있는데, 이러한 **주기적 신호들을 다루는 역할**을 한다.
        - 이를 위해 1차원 데이터를 주기성을 반영하여 세로 길이가 각기 다른 **2차원 데이터로 변환(Reshape)**하는 과정을 거친다.
    - **Multi-Scale Discriminator(MSD)**: 오디오의 **연속적인 패턴을 파악**하고, **장기 의존성을 반영**하는 역할을 한다.
        - 이를 위해 연속적인 Waveform 데이터를 여러 배수의 **Down Sampling**을 거쳐 input data로 활용한다.

- Generator와 Discriminator를 조금 더 자세히 살펴보면 다음과 같다.
<img src="3.png" width=70% height=70%>

    - **Generator**는 다른 GAN의 Generator처럼 **Upsampling**하는 과정을 여러 번 거친다.
        - Upsampling에는 **Transposed Convolution**을 이용한다.
        - **Multi-Receptive Field(MRF)**라는 특수한 Module을 이용하여 오디오의 다양한 길이의 패턴을 관찰한다.
    - **MPD**의 **Reshape**는 주기 p를 소수인 2, 3, 5, 7, 11로 조절하여 총 5개의 input을 만들고, 5개의 sub MPD를 만든다.
        - Feature를 추출할 때에는 Nx1 Conv2d를 이용한다.
    - **MSD**의 **Down Sampling**에는 **Average Pooling**을 이용하는데, 여기에서는 Average Pooling을 0번, 1번, 2번 거친 waveform 총 3개의 input을 만들고, 3개의 sub MSD를 만든다.
        - Average Pooling을 거칠 때마다 **2배 Down Sampling**이 된다.
        - Feature를 추출할 때에는 Grouped Conv1d를 이용한다.

## 2.2. Generator

- **Generator**의 내부 구조는 다음과 같다.
<img src="4.png" width=70% height=70%>

    - **Multi-Receptive Field Fusion(MRF)**: Residual Connection과 Dilation Convolution을 가진 **Residual Block 3개**로 이루어져 있다.
        - 각각의 Residual Block은 Conv1d의 **kernel 사이즈를 다르게 설정**하여 **다양한 패턴을 관찰**할 수 있다.
        - **Dilation convolution을 활용**했기 때문에 **넓은 수용 영역(Receptive Field)**을 가질 수 있다.

- Generator의 추가적인 구조와 Input, Features, Output의 shape은 다음과 같다.
<img src="5.png" width=70% height=70%>

    - **Transposed Convolution**: 2개의 8배 Upsampling과 2개의 2배 Upsampling으로 Mel-spectrogram은 최종적으로 256배 더 잘게 쪼개진다.
    - 마지막 Conv1d의 channel이 1이므로 Output의 channel 수가 1이 되어 정상적으로 waveform을 형성하게 된다.

## 2.3. Discriminator

### 2.3.1. Multi-Period Discriminator(MPD)

- **MPD**의 구조는 다음과 같다.
<img src="6.png" width=70% height=70%>

    - Reshape 주기에 따라 총 5개의 Sub MPD가 생성된다.
    - **Reflect mode**: padding 방법 중 하나로, 경계 부분에 대칭 반사하여 padding한다.
        - 예) [1, 2, 3, 4, 5]를 뒤쪽으로 2개 padding한다면 [1, 2, 3, 4, 5, 4, 3]이 된다.
        - 장점: 경계 부분이 자연스럽게 보존된다.
    - Sub MPD에서는 Conv2d로 고급 정보를 담고 있는 features를 생성해내게 된다.
        - 모든 Convolution의 Weight에는 **weight normalization**을 한다.
    - Real/Fake를 판단할 때 마지막에 Sigmoid 함수가 필요하지 않다. 그 이유는 Discriminator의 Loss를 적절히 변형해주었기 때문이다. (3절 참고)

### 2.3.2. Multi-Scale Discriminator

- **Multi-Scale Discriminator(MSD)**의 구조는 다음과 같다.
<img src="7.png" width=70% height=70%>

    - Down Sampling에 따라 총 3개의 Sub MSD가 생성된다.
    - sub MSD에서는 파라미터 수를 크게 줄이고자 **Grouped Convolution**를 이용한다.
        - **Grouped Convolution**
        <img src="8.png" width=70% height=70%>
        
        - Grouped Convolution은 1) **파라미터 수를 크게 줄이고**, 2) **각 그룹마다 서로 다른 특성을 학습**하도록 할 수 있다.
    - 모든 Convolution 연산에는 weight normalization을 해주게 되는데, 첫 번째 sub MSD에는 **spectral normalization**을 이용한다.
        - 이는 안정적으로 학습을 시키도록 도와준다.
        
    - 위 구조는 MelGAN 논문에서 가져왔다.

## 2.4. Training Loss Terms

### 2.4.1. GAN Loss

- Generator GAN Loss는 LS-GAN의 Loss를 따르며, 다음과 같다.
$$\mathcal{L}_{Adv}(G;D) = \mathbb{E}_{s} \left[ (D(G(s))-1)^{2} \right]$$
($\mathbb{E}$: Expectation, $G$: Generator, $D$: Discriminator, $x$: real waveform, $s$: generated waveform)
    - LS-GAN 논문에 따르면 위 Loss는 아래의 원래 GAN의 Loss를 최소화하는 것과 원리가 같음을 증명한다.
    $$\mathcal{L}_{Adv}(G;D) = \mathbb{E}_{s} \left[ \log(D(G(s))-1) \right]$$
    - LS-GAN의 Loss는 **$D(G(s))$의 값이 반드시 0과 1사이일 필요가 없다.**
    - LS-GAN의 Loss는 훈련의 안정화에 기여한다.

- Discriminator GAN Loss는 다음과 같다.
$$\mathcal{L}_{Adv}(G;D) = \mathbb{E}_{(x, s)} \left[ (D(x) - 1)^{2} + (D(G(s)))^{2} \right]$$
    - LS-GAN 논문에 따르면 위 Loss는 아래의 원래 GAN의 Loss를 최소화하는 것과 원리가 같음을 증명한다.
    $$\mathcal{L}_{Adv}(G;D) = \mathbb{E}_{(x, s)} \left[ \log(D(x) - 1) + \log(D(G(s))) \right]$$

### 2.4.2. Feature Matching Loss

- **Feature Matching Loss**는 실제 waveform과 생성된 waveform에 대하여 Discriminator를 통과시킬 때 모든 features끼리의 차이를 Loss로 설정한 것이다.  
    - 즉, 실제 waveform과 생성된 waveform에 대한 Discriminator의 Features가 서로 같도록 학습한다.
- Feature Matching Loss의 식은 다음과 같다.
$$\mathcal{L}_{FM}(G;D) = \mathbb{E}_{(x, s)} \left[ \sum_{i=1}^{T} \frac{1}{N_{i}} ||D^{i}(x) - D^{i}(G(s))||_{1} \right]$$
($T$: Discriminator의 층의 개수, $D^{i}$: $i$번째 층의 features, $N_{i}$: $i$번째 층의 features의 개수)

### 2.4.3. Mel-Spectrogram Loss

- **Mel-Spectrogram Loss**는 실제 waveform의 mel-spectrogram과 생성된 waveform의 mel-spectrogram의 차이를 Loss로 설정한 것이다.
- Mel-Spectrogram Loss의 식은 다음과 같다.
$$\mathcal{L}_{Mel}(G) = \mathbb{E}_{(x, s)} \left[ ||\phi(x) - \phi(G(x))||_{1} \right]$$
($\phi$: waveform을 mel-spectrogram으로 변환하는 함수)
    - **Mel-Spectrogram Loss**는 HiFi-GAN에서 **상당히 중요한 역할**을 하며, 그 이점은 다음과 같다.
        1. 현실적인 waveform을 합성하도록 도와준다.
        2. 초기 적대적 훈련의 안정화에 도움이 된다.

### 2.4.4. Final Loss

- Generator의 최종 Loss는 다음과 같다.
$$ \mathcal{L}_{G}= \sum_{k=1}^{K} \left[ \mathcal{L}_{Adv}(G;D_{k}) + \lambda_{fm} \mathcal{L}_{FM}(G;D_{k}) \right] + \lambda_{mel} \mathcal{L}_{Mel}(G)$$
($\lambda_{fm}$ = 2, $\lambda_{mel}$ = 45)
- Discriminator의 최종 Loss는 다음과 같다.
$$ \mathcal{L}_{D} = \sum_{k=1}^{K} \mathcal{L}_{Adv}(D_{k};G)$$

**[참고 자료]**
1. HiFi-GAN 논문: https://arxiv.org/abs/2010.05646
2. HiFi-GAN 논문 저자의 구현: https://github.com/jik876/hifi-gan
3. Weight Normalization 논문: https://arxiv.org/abs/1602.07868
4. Spectral Normalization 논문: https://arxiv.org/abs/1802.05957
5. GAN 논문: https://arxiv.org/abs/1406.2661
6. LS-GAN 논문: https://arxiv.org/abs/1611.04076